In [14]:
import os
import json
import pandas as pd
from tqdm.notebook import tqdm
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import torch 

In [15]:
root_path = Path('./data/BO')
n_optim = 50
results_bo = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            #print(os.path.join(subdir, file))
            _, _, ds, query, budget, params, seed  = subdir.split('/')

            if ds not in results_bo:
                results_bo[ds] = {}

            if query not in results_bo[ds]:
                results_bo[ds][query] = {}

            if budget not in results_bo[ds][query]:
                results_bo[ds][query][budget] = {}

            if params not in results_bo[ds][query][budget]:
                results_bo[ds][query][budget][params] = {}

            if seed not in results_bo[ds][query][budget][params]:
                results_bo[ds][query][budget][params][seed] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results_bo[ds][query][budget][params][seed] = res

In [16]:
test_accuracys_bo = {}
for ds in results_bo:
    test_accuracys_bo[ds] = {}
    for query in results_bo[ds]:
        test_accuracys_bo[ds][query] = {}
        for budget in results_bo[ds][query]:
            test_accuracys_bo[ds][query][budget] = {}
            for params in results_bo[ds][query][budget]:
                test_accuracys_bo[ds][query][budget][params] = []
                for seed in results_bo[ds][query][budget][params]:
                    # Extract final test acc from json file
                    final_test_acc = results_bo[ds][query][budget][params][seed]['test_stats']['test_acc']
                    test_accuracys_bo[ds][query][budget][params].append(final_test_acc*100)

In [17]:
# Validate that each param group has 3 finished runs
unfinished_runs = []
for ds in test_accuracys_bo:
    for query in test_accuracys_bo[ds]:
        for budget in test_accuracys_bo[ds][query]:
            for params in test_accuracys_bo[ds][query][budget]:
                if len(test_accuracys_bo[ds][query][budget][params]) != 3:
                    print("Assertion: The run on ds "+str(ds)+" with query strategy "+str(query)+ " and parameters "+params+" and budget "+budget+" has not had 3 finished runs yet and \n will be left out for the following result table!")
                    unfinished_runs.append([ds, query, budget, params])
                
# Remove unfinished runs
for ds, query, budget, params in unfinished_runs:
    test_accuracys_bo[ds][query][budget].pop(params)

Assertion: The run on ds CIFAR100 with query strategy badge and parameters lr0.001_wd0.05 and budget budget4000 has not had 3 finished runs yet and 
 will be left out for the following result table!
Assertion: The run on ds CIFAR100 with query strategy badge and parameters lr0.01_wd0.0005 and budget budget4000 has not had 3 finished runs yet and 
 will be left out for the following result table!


In [22]:
root_path = Path('./data/AL')
n_optim = 50
results_al = {}

for subdir, dirs, files in os.walk(root_path):
    for file in files:
        if file == 'results.json':
            #print(os.path.join(subdir, file))
            _, _, ds, query, params, seed  = subdir.split('/')

            if ds not in results_al:
                results_al[ds] = {}

            if query not in results_al[ds]:
                results_al[ds][query] = {}

            if params not in results_al[ds][query]:
                results_al[ds][query][params] = {}

            if seed not in results_al[ds][query][params]:
                results_al[ds][query][params][seed] = {}

            with open(subdir + '/' + 'results.json', 'r') as f:
                res = json.load(f)
            results_al[ds][query][params][seed] = res

In [28]:
results_al[ds][query][params][seed]['cycle0']['test_stats']

#len(results_al[ds][query][params][seed].keys())

{'num_labeled_samples': 100, 'accuracy': 0.2734000086784363}

In [29]:
test_accuracys_al = {}
for ds in results_al:
    test_accuracys_al[ds] = {}
    for query in results_al[ds]:
        test_accuracys_al[ds][query] = {}
        for budget in ['budget2000', 'budget4000']:
            test_accuracys_al[ds][query][budget] = {}
            for params in results_al[ds][query]:
                test_accuracys_al[ds][query][budget][params] = []
                for seed in results_al[ds][query][params]:
                    # Extract final test acc from json file
                    test_acc = results_al[ds][query][params][seed]['cycle20' if budget == 'budget2000' else 'cycle39']['test_stats']['accuracy']
                    test_accuracys_al[ds][query][budget][params].append(test_acc*100)

In [33]:
test_accuracys_al

{'CIFAR10': {'badge': {'budget2000': {'lr0.01_wd0.005': [79.08999919891357,
     76.88000202178955,
     77.38000154495239,
     76.95000171661377,
     77.42999792098999],
    'lr0.001_wd0.005': [67.5599992275238,
     67.61000156402588,
     67.76999831199646,
     67.03000068664551,
     67.58000254631042],
    'lr0.001_wd0.05': [78.39999794960022,
     74.75000023841858,
     77.42999792098999,
     74.94999766349792,
     77.28999853134155],
    'lr0.001_wd0.0005': [65.60999751091003,
     65.9600019454956,
     66.3100004196167,
     65.34000039100647,
     65.23000001907349],
    'lr0.01_wd0.0005': [73.50000143051147,
     70.70000171661377,
     72.67000079154968,
     70.91000080108643,
     74.15000200271606],
    'lr0.01_wd0.05': [59.289997816085815,
     69.38999891281128,
     57.12000131607056,
     64.20999765396118,
     59.2199981212616],
    'lr0.1_wd0.005': [70.41000127792358,
     70.85999846458435,
     70.69000005722046,
     66.60000085830688,
     68.43000054359

In [57]:
experiments = ['BO', 'AL']
datasets = ['CIFAR10', 'CIFAR100']
budgets = ['budget2000', 'budget4000']
querys = ['random', 'entropy', 'coreset', 'badge']
parameters = ['lr0.001_wd0.05', 'lr0.01_wd0.0005', 'lr0.01_wd0.05', 'lr0.1_wd0.005']

string = ""

for ds in datasets:
    string += "########################################################"+ds+"############################################################# \n"
    for budget in budgets:
        string += "-----------------------------------------------------" + budget + "----------------------------------------------------- \n"
        string += "          | **AL**       | **BO**        | **AL**        | **BO**        | **AL**        | **BO**        | **AL**        | **BO**        | \n"
        for query in querys:
            string += "| " + query
            if query == 'random' : 
                string += "  |"
            elif query == 'badge':
                string += "   |"
            else:
                string += " |"
            for params in parameters:
                if params in test_accuracys_al[ds][query][budget] and params in test_accuracys_bo[ds][query][budget]:
                    string += f"{np.mean(test_accuracys_al[ds][query][budget][params]):05.2f}" + " ± " + f"{np.std(test_accuracys_al[ds][query][budget][params]):05.2f}"
                    string += " | "
                    string += f"{np.mean(test_accuracys_bo[ds][query][budget][params]):05.2f}" + " ± " + f"{np.std(test_accuracys_bo[ds][query][budget][params]):05.2f}"
                    string += " | "
            string += "\n"
                    
print(string)

########################################################CIFAR10############################################################# 
-----------------------------------------------------budget2000----------------------------------------------------- 
          | **AL**       | **BO**        | **AL**        | **BO**        | **AL**        | **BO**        | **AL**        | **BO**        | 
| random  |75.11 ± 01.03 | 76.96 ± 00.43 | 70.76 ± 00.64 | 77.72 ± 00.17 | 68.04 ± 04.19 | 77.14 ± 00.59 | 71.68 ± 00.97 | 77.33 ± 00.22 | 
| entropy |75.45 ± 00.33 | 77.63 ± 00.34 | 72.25 ± 00.83 | 77.78 ± 00.76 | 35.39 ± 07.59 | 77.12 ± 00.24 | 33.49 ± 18.12 | 76.61 ± 01.17 | 
| coreset |76.62 ± 00.55 | 77.42 ± 00.09 | 71.02 ± 00.94 | 76.47 ± 00.83 | 46.45 ± 10.50 | 77.11 ± 02.07 | 67.81 ± 05.19 | 76.98 ± 00.88 | 
| badge   |76.56 ± 01.45 | 78.56 ± 00.44 | 72.39 ± 01.38 | 78.91 ± 00.48 | 61.85 ± 04.43 | 77.61 ± 00.61 | 69.40 ± 01.65 | 78.27 ± 01.37 | 
-----------------------------------------------------bud

In [55]:
print('{:05}'.format(1.3))

001.3
